# Application On all sources

## Imports

In [6]:
%load_ext autoreload
%autoreload 2
import numpy as np 
from matplotlib import pyplot as plt 
import pandas as pd 
import seaborn as sns
import joblib


In [7]:
from utilities import make_model

## Load

### Load Model

In [8]:
model = joblib.load('models/lightGBM-final.pkl')
model